# OsmGt example

## Prepare input parameters

In [1]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show

import geopandas as gpd

from shapely.geometry import Point
from shapely.ops import linemerge

from easy_map_bokeh import EasyMapBokeh

from osmgt import OsmGt

from graph_tool.topology import shortest_path


output_notebook()


location = "Roanne"


Loading BokehJS ...

## Get POIs

In [2]:
%%time

pois_from_location_gdf = OsmGt.pois_from_location(location).get_gdf()

2020-09-04 21:19:30 - OsmGtPoi        - INFO     : From location: Roanne
2020-09-04 21:19:30 - OsmGtPoi        - INFO     : Loading data...
2020-09-04 21:19:30 - OsmGtPoi        - INFO     : NominatimApi: Query 200:OK in 0.27993 sec.
2020-09-04 21:19:31 - OsmGtPoi        - INFO     : OverpassApi: Query 200:OK in 0.759569 sec.
2020-09-04 21:19:31 - OsmGtPoi        - INFO     : Formating data
2020-09-04 21:19:31 - OsmGtPoi        - INFO     : Prepare GeoDataframe
2020-09-04 21:19:31 - OsmGtPoi        - INFO     : GeoDataframe Ready
CPU times: user 99 ms, sys: 12.5 ms, total: 111 ms
Wall time: 1.14 s


## Get Roads

In [3]:
%%time
roads_from_location = OsmGt.roads_from_location(
    location,
    mode="vehicle",
    additional_nodes=pois_from_location_gdf
)
roads_from_location_gdf = roads_from_location.get_gdf()

2020-09-04 21:19:31 - OsmGtRoads      - INFO     : From location: Roanne
2020-09-04 21:19:31 - OsmGtRoads      - INFO     : Loading data...
2020-09-04 21:19:31 - OsmGtRoads      - INFO     : NominatimApi: Query 200:OK in 0.140164 sec.
2020-09-04 21:19:32 - OsmGtRoads      - INFO     : OverpassApi: Query 200:OK in 1.307931 sec.
2020-09-04 21:19:32 - OsmGtRoads      - INFO     : Rebuild network data
2020-09-04 21:19:32 - OsmGtRoads      - INFO     : Network cleaning...
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Starting: Adding new nodes on the network
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Find nearest line for each node
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Split line
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Topology lines checker: to add: 184, to split: 182
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Starting: Find intersections
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Done: Find intersections
2020-09-04 21:19:33 - OsmGtRoads

## Check topology rebuilt

In [4]:
%%time

roads_topology_gdf = roads_from_location.topology_checker()

lines_unchanged = roads_topology_gdf["lines_unchanged"]
lines_added = roads_topology_gdf["lines_added"]
lines_split = roads_topology_gdf["lines_split"]
nodes_added = roads_topology_gdf["nodes_added"]
intersections_added = roads_topology_gdf["intersections_added"]

layers_to_add = [
    {
        "input_gdf": lines_unchanged,
        "legend": "roads unchanged",
        "color": "black",
    },
    {
        "input_gdf": lines_added,
        "legend": "roads added",
        "color": "green",
    },
    {
        "input_gdf": lines_split,
        "legend": "roads split",
        "color": "orange",
    },
    {
        "input_gdf": intersections_added,
        "legend": "Intersections added",
        "color": "brown",
    },
    {
        "input_gdf": nodes_added,
        "legend": "Nodes added",  # POIs here
        "style": "square",
        "color": "blue",
        "size": 9
    },
]

my_map = EasyMapBokeh(
    "My roads and POIs",
    layers=layers_to_add
)
show(my_map.figure)

2020-09-04 21:19:33 - OsmGtRoads      - INFO     : Prepare topology data
2020-09-04 21:19:33 - OsmGtRoads      - INFO     : GeoDataframe Ready


CPU times: user 3.35 s, sys: 40.6 ms, total: 3.4 s
Wall time: 3.43 s


## Get network and display it

In [5]:
%%time
graph = roads_from_location.get_graph()

# graph.plot()


2020-09-04 21:19:37 - OsmGtRoads      - INFO     : Prepare graph
CPU times: user 2.03 s, sys: 11.4 ms, total: 2.04 s
Wall time: 2.16 s


## Compute a shortest path

## Compute a shortest path

In [6]:
%%time
# now, we have to define a start point and a end point and get their wkt
start_node_topo_uuid = 47
end_node_topo_uuid = 63

# 'topo_uuid' is generated by osmgt (during the topology processing).
# Some roads has been split that's whyso this id has been created.
start_node_gdf = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == start_node_topo_uuid]
end_node_gdf = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == end_node_topo_uuid]

start_node_wkt = start_node_gdf.iloc[0]["geometry"].wkt
end_node_wkt = end_node_gdf.iloc[0]["geometry"].wkt

# the graph have some methods (graph-tools method always exists!) to find egdes, vertices... Let's use the .find_vertex_from_name(). the wkt is the vertex name...
source_vertex = graph.find_vertex_from_name(start_node_wkt)
target_vertex = graph.find_vertex_from_name(end_node_wkt)

# shortest path computing...
path_vertices, path_edges = shortest_path(
    graph,
    source=source_vertex,
    target=target_vertex,
    weights=graph.edge_weights  # weigth is based on line length
)

# get path by using edge names
roads_ids = [
    graph.edge_names[edge]
    for edge in path_edges
]

roads_gdf_copy = roads_from_location_gdf.copy(deep=True)
shortest_path_found = roads_gdf_copy[roads_from_location_gdf['topo_uuid'].isin(roads_ids)].to_crs(3857)['geometry'].to_list()
shortest_path_found_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:3857", geometry=[linemerge(shortest_path_found)])

layers_to_add = [
    {
        "input_gdf": shortest_path_found_gdf,
        "legend": "shortest_path",
        "color": "red",
        "line_width": 5
    },
    {
        "input_gdf": start_node_gdf,
        "legend": "source node",
        "color": "blue",
        "style": "circle",
        "size": 9
    },
    {
        "input_gdf": end_node_gdf,
        "legend": "target node",
        "color": "green",
        "style": "circle",
        "size": 9
    },
]

my_shortest_path_map = EasyMapBokeh(
    "My shortest path",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

CPU times: user 236 ms, sys: 5.82 ms, total: 242 ms
Wall time: 248 ms


### By using OsmGt function

In [7]:
%%time

start_node_topo_uuid = 47
end_node_topo_uuid = 63

start_node_gdf = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == start_node_topo_uuid]
end_node_gdf = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == end_node_topo_uuid]

start_node = start_node_gdf.iloc[0]["geometry"]
end_node = end_node_gdf.iloc[0]["geometry"]

shortest_paths = OsmGt.shortest_path_from_location(
    "Roanne",
    [
        (start_node, end_node),
        (start_node, end_node),
        (start_node, end_node),
        (start_node, end_node),
        (start_node, end_node),
        (start_node, end_node)
    ],
    mode="pedestrian"
)
layers_to_add = [
    {
        "input_gdf": shortest_paths[["geometry"]],
        "legend": "shortest_path",
        "color": "red",
        "line_width": 5
    },
    {
        "input_gdf": start_node_gdf,
        "legend": "source node",
        "color": "blue",
        "style": "circle",
        "size": 9
    },
    {
        "input_gdf": end_node_gdf,
        "legend": "target node",
        "color": "green",
        "style": "circle",
        "size": 9
    },
]

my_shortest_path_map = EasyMapBokeh(
    "My shortest path",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

2020-09-04 21:19:39 - OsmGtShortestPath - INFO     : Shortest path processing...
2020-09-04 21:19:39 - OsmGtShortestPath - INFO     : From location: Roanne
2020-09-04 21:19:39 - OsmGtShortestPath - INFO     : Loading data...
2020-09-04 21:19:39 - OsmGtShortestPath - INFO     : NominatimApi: Query 200:OK in 0.148434 sec.
2020-09-04 21:19:40 - OsmGtShortestPath - INFO     : OverpassApi: Query 200:OK in 0.912069 sec.
2020-09-04 21:19:40 - OsmGtShortestPath - INFO     : Rebuild network data
2020-09-04 21:19:40 - OsmGtShortestPath - INFO     : Network cleaning...
2020-09-04 21:19:40 - OsmGtShortestPath - INFO     : Starting: Adding new nodes on the network
2020-09-04 21:19:40 - OsmGtShortestPath - INFO     : Find nearest line for each node
2020-09-04 21:19:41 - OsmGtShortestPath - INFO     : Split line
2020-09-04 21:19:41 - OsmGtShortestPath - INFO     : Topology lines checker: to add: 2, to split: 2
2020-09-04 21:19:41 - OsmGtShortestPath - INFO     : Starting: Find intersections
2020-09-0

CPU times: user 1.96 s, sys: 27.6 ms, total: 1.99 s
Wall time: 3.21 s


## Compute an isochrone

### Isochrone from times

In [8]:
%%time
source_node = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == start_node_topo_uuid]

isochrones_polygon_values = {
    2: "#d9ef8b",
    5: "#fee08b",
    10: "#f46d43"
}
# 2 = 2 min ; 5 = 5 min ; 10 = 10 min


isochrones_lines_values = {
    2: "#668100",
    5: "#e2a803",
    10: "#962603"
}

trip_speed = 3  # km/h

location_point = source_node.iloc[0]["geometry"]
isochrones_polygons_from_location, isochrones_lines_from_location = OsmGt.isochrone_from_source_node(
    location_point,
    list(isochrones_polygon_values.keys()),
    trip_speed,
    mode="pedestrian"
)

isochrones_polygons_from_location["color"] = isochrones_polygons_from_location["iso_name"].map(lambda x: isochrones_polygon_values[int(x.split(" ")[0])])
isochrones_lines_from_location["color"] = isochrones_lines_from_location["iso_name"].map(lambda x: isochrones_lines_values[int(x.split(" ")[0])])


layers_to_add = [
    {
        "input_gdf": isochrones_polygons_from_location,
        "legend": "iso_name",
        "fill_color": "color",
        "line_color": "color",
        "fill_alpha": 0.5
    },
    {
        "input_gdf": source_node,
        "legend": "Source node",
        "style": "circle",
        "color": "red",
        "size": 5
    },
    {
        "input_gdf": isochrones_lines_from_location,
        "legend": "iso_name",
        "color": "color",
        "line_width": 2
    },
]

my_shortest_path_map = EasyMapBokeh(
    "isochrone",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

2020-09-04 21:19:42 - OsmGtIsochrone  - INFO     : Isochrone processing...
2020-09-04 21:19:42 - OsmGtIsochrone  - INFO     : From bbox: (4.0629714353691035, 46.03634090608913, 4.076446164630895, 46.045694298051714)
2020-09-04 21:19:42 - OsmGtIsochrone  - INFO     : Loading data...
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : OverpassApi: Query 200:OK in 0.307222 sec.
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Rebuild network data
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Network cleaning...
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Starting: Adding new nodes on the network
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Find nearest line for each node
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Split line
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Topology lines checker: to add: 1, to split: 1
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Starting: Find intersections
2020-09-04 21:19:43 - OsmGtIsochrone  - INFO     : Done: Find in

ERROR:bokeh.core.validation.check:E-1006 (NON_MATCHING_DATA_SOURCES_ON_LEGEND_ITEM_RENDERERS): LegendItem.label is a field, but renderer data sources don't match: LegendItem(id='1704', ...)


CPU times: user 4.52 s, sys: 90.8 ms, total: 4.61 s
Wall time: 5 s


In [9]:
isochrones_polygons_from_location.to_file("hello.geojson", driver="GeoJSON")

### Isochrone from distances

In [10]:
%%time
source_node = pois_from_location_gdf[pois_from_location_gdf['topo_uuid'] == start_node_topo_uuid]

trip_speed = 3  # km/h

isochrones_polygon_values = {
    500: "#d9ef8b",
    750: "#fee08b",
    1000: "#f46d43"
}
# 2 = 2 min ; 5 = 5 min ; 10 = 10 min


isochrones_lines_values = {
    500: "#668100",
    750: "#e2a803",
    1000: "#962603"
}

location_point = source_node.iloc[0]["geometry"]
isochrones_polygons_from_location, isochrones_lines_from_location = OsmGt.isochrone_distance_from_source_node(
    location_point,
    list(isochrones_polygon_values.keys()),  # meters
    trip_speed,
    mode="pedestrian"
)

isochrones_polygons_from_location["color"] = isochrones_polygons_from_location["iso_distance"].map(lambda x: isochrones_polygon_values[x])
isochrones_lines_from_location["color"] = isochrones_lines_from_location["iso_distance"].map(lambda x: isochrones_lines_values[x])


layers_to_add = [
    {
        "input_gdf": isochrones_polygons_from_location,
        "legend": "iso_distance",
        "fill_color": "color",
        "line_color": "color",
        "fill_alpha": 0.5
    },
    {
        "input_gdf": isochrones_lines_from_location,
        "legend": "iso_distance",
        "color": "color",
        "line_width": 2
    },
    {
        "input_gdf": source_node,
        "legend": "Source node",
        "style": "circle",
        "color": "orange",
        "size": 15
    },
]

my_shortest_path_map = EasyMapBokeh(
    "isochrone",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)


2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Isochrone processing...
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : From bbox: (4.056234070738207, 46.031663616316784, 4.083183529261792, 46.050370400246656)
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Loading data...
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : OverpassApi: Query 200:OK in 0.268086 sec.
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Rebuild network data
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Network cleaning...
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Starting: Adding new nodes on the network
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Find nearest line for each node
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Split line
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Topology lines checker: to add: 1, to split: 1
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Starting: Find intersections
2020-09-04 21:19:48 - OsmGtIsochrone  - INFO     : Done: Find in

ERROR:bokeh.core.validation.check:E-1006 (NON_MATCHING_DATA_SOURCES_ON_LEGEND_ITEM_RENDERERS): LegendItem.label is a field, but renderer data sources don't match: LegendItem(id='1918', ...)


CPU times: user 19.9 s, sys: 155 ms, total: 20 s
Wall time: 20.4 s


In [11]:
isochrones_polygons_from_location.to_file("hello.geojson", driver="GeoJSON")